In [48]:
import requests
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [49]:
def fetch_aq(city):
    end_time = pd.datetime.now().strftime('%Y-%m-%d-%H')
    r = requests.get("https://biendata.com/competition/airquality/" + city + "/2018-03-30-0/" + end_time + "/2k0d1d8")
    df = pd.DataFrame([i.decode('utf8').split(',') for i in r.content.splitlines()])
    df.columns = df.iloc[0]
    df = df.reindex(df.index.drop(0))
    del df['id']
    df.columns = ['stationId', 'utc_time', 'PM2.5', 'PM10', 'NO2', 'CO', 'O3', 'SO2']
    return df

df_aq = pd.read_csv("../input/beijing_1701_1803.csv")
df_aq_new = fetch_aq("bj")
df_station = pd.read_csv("../input/Beijing_AirQuality_Stations.csv")
df_meo = pd.read_csv("../input/Beijing_historical_meo_grid.csv")

df_aq = df_aq[df_aq.utc_time< '2018-03-31 07:00:00']
df_aq = pd.concat([df_aq, df_aq_new])


# df_aq.utc_time = pd.to_datetime(df_aq.utc_time)
# df_meo.utc_time = pd.to_datetime(df_meo.utc_time)

# target_cols = ['PM2.5', 'PM10', 'O3']
# dfs = []
# for stationId in df_aq.stationId.unique():
#     df = df_aq[df_aq['stationId']==stationId]
#     df = df.resample('1H', on='utc_time').sum()
#     df = df.reset_index()
#     df['stationId'] = stationId
#     dfs.append(df)
# df_origin = pd.concat(dfs)

# # Station
# idx_to_station = {k:v for k,v in df_station.Station_ID.iteritems()}
# near_stations = []
# for k, row in df_station.iterrows():
#     result =  np.sqrt((111*(df_station.longitude-row[1]))**2 + (74*(df_station.latitude-row[2]))**2).nsmallest(4)
#     near_idx = np.sqrt((111*(df_station.longitude-row[1]))**2 + (74*(df_station.latitude-row[2]))**2).nsmallest(4).index[1:4].tolist()
#     near_station = [idx_to_station[idx] for idx in near_idx]
#     near_stations.append(near_station)
#     print(k, row[0])
#     print(result, "\n")
    
# df_station_near = pd.concat([df_station, pd.DataFrame(near_stations, columns=['near_1', 'near_2', 'near_3'])], axis=1 )

In [3]:
# Spatial 
def spatial_step(df, target):
    for near in nears:
        df = pd.merge(df, df[[target, 'stationId','utc_time']], 
                     how='left', suffixes=('', '_'+ near),
                     left_on=['utc_time', near], 
                     right_on=['utc_time', 'stationId'])
    return df

# Temperial 
def temporal_step(df, target):
    dfs_ = []
    nulls = []
    for station in df.stationId.unique():
        df_ = df[df.stationId==station]
        df_ = df_.sort_values('utc_time')

        for i in [-2, -1, 1, 2]:
            df_[target + "_t{0:+}".format(i)] = df_[target].shift(i*-1)

        cols = [target + "_t{0:+}".format(i) for i in [-2, -1, 1, 2]]
        cols += [target+'_near_1', target+'_near_2', target+'_near_3', target, 'utc_time']

        df_[target+"_pred"] = df_[cols].mean(axis=1)

        # Update valie
        null_idxs = station_to_null_idxs[station]
        if EVALUATION:
            eval_idxs = station_to_eval_idxs[station]
        df_.loc[null_idxs, target] = df_[target + '_pred']
        if EVALUATION:
            df_.loc[eval_idxs, target] = df_[target + '_pred']

        for i in [-2, -1, 1, 2]:
            df_[target + "_t{0:+}".format(i)] = df_[target].shift(i*-1)
        null = df_[df_[target].isnull()].shape[0]
        nulls.append(null)
#         print(N_null, station, target)
        dfs_.append(df_[['utc_time', 'stationId', target]])
    return dfs_, nulls

df_stKNNs = []
EVALUATION = False
for target in target_cols:
    print(target)
    station_to_null_idxs = {}
    station_to_eval_idxs = {}
    MSEs = []

    nears = ['near_1', 'near_2', 'near_3']

    df_impute = pd.merge(df_origin, df_station_near, 'left', left_on='stationId', right_on='Station_ID')
    for station in df_impute.stationId.unique():
        df_ = df_impute[df_impute.stationId==station]
        df_ = df_.sort_values('utc_time')
        station_to_null_idxs[station] = df_[target].isnull() # for params update
        if EVALUATION:
            eval_condition = (df_.utc_time>='2017-05-01 00:00:00') & (df_.utc_time<='2017-05-01 23:00:00') & (df_.stationId=="tiantan_aq") 
            station_to_eval_idxs[station] = eval_condition # for evaluation

    N_nulls = 999
    while N_nulls>0:
        df_impute = spatial_step(df_impute, target)
        df_imputes, nulls = temporal_step(df_impute, target)
        df_impute = pd.concat(df_imputes)
        df_impute = pd.merge(df_impute, df_station_near, 'left', left_on='stationId', right_on='Station_ID')
        N_nulls = sum(nulls)
        print(N_nulls, nulls)
    
    df_stKNNs.append(df_impute)
    


18613 [527, 529, 544, 529, 524, 525, 525, 557, 525, 529, 529, 524, 530, 561, 524, 524, 548, 547, 526, 524, 524, 528, 527, 526, 531, 524, 535, 528, 531, 524, 526, 551, 529, 527, 551]
16119 [458, 458, 462, 461, 457, 457, 458, 479, 457, 459, 460, 457, 461, 479, 457, 457, 468, 461, 457, 457, 457, 458, 460, 458, 460, 457, 461, 457, 463, 457, 457, 477, 461, 459, 457]
14931 [426, 425, 427, 425, 425, 425, 426, 435, 425, 427, 428, 425, 428, 435, 425, 425, 427, 425, 425, 425, 425, 425, 428, 426, 425, 425, 428, 425, 431, 425, 425, 432, 425, 427, 425]
14205 [406, 405, 407, 405, 405, 405, 406, 410, 405, 405, 405, 405, 407, 411, 405, 405, 407, 405, 405, 405, 405, 405, 405, 406, 405, 405, 407, 405, 407, 405, 405, 411, 405, 405, 405]
13711 [392, 391, 392, 391, 391, 391, 391, 395, 391, 391, 391, 391, 393, 397, 391, 391, 393, 391, 391, 391, 391, 391, 391, 391, 391, 391, 393, 391, 393, 391, 391, 397, 391, 391, 391]
13291 [380, 379, 380, 379, 379, 379, 379, 383, 379, 379, 379, 379, 381, 385, 379, 379, 381

1901 [54, 54, 55, 54, 54, 54, 54, 54, 54, 54, 54, 54, 56, 56, 54, 54, 56, 54, 54, 54, 54, 54, 54, 54, 54, 54, 56, 54, 54, 54, 54, 56, 54, 54, 54]
1761 [50, 50, 51, 50, 50, 50, 50, 50, 50, 50, 50, 50, 52, 52, 50, 50, 52, 50, 50, 50, 50, 50, 50, 50, 50, 50, 52, 50, 50, 50, 50, 52, 50, 50, 50]
1621 [46, 46, 47, 46, 46, 46, 46, 46, 46, 46, 46, 46, 48, 48, 46, 46, 48, 46, 46, 46, 46, 46, 46, 46, 46, 46, 48, 46, 46, 46, 46, 48, 46, 46, 46]
1481 [42, 42, 43, 42, 42, 42, 42, 42, 42, 42, 42, 42, 44, 44, 42, 42, 44, 42, 42, 42, 42, 42, 42, 42, 42, 42, 44, 42, 42, 42, 42, 44, 42, 42, 42]
1341 [38, 38, 39, 38, 38, 38, 38, 38, 38, 38, 38, 38, 40, 40, 38, 38, 40, 38, 38, 38, 38, 38, 38, 38, 38, 38, 40, 38, 38, 38, 38, 40, 38, 38, 38]
1201 [34, 34, 35, 34, 34, 34, 34, 34, 34, 34, 34, 34, 36, 36, 34, 34, 36, 34, 34, 34, 34, 34, 34, 34, 34, 34, 36, 34, 34, 34, 34, 36, 34, 34, 34]
1061 [30, 30, 31, 30, 30, 30, 30, 30, 30, 30, 30, 30, 32, 32, 30, 30, 32, 30, 30, 30, 30, 30, 30, 30, 30, 30, 32, 30, 30, 30

4738 [134, 134, 135, 134, 136, 139, 134, 138, 134, 134, 134, 134, 137, 140, 134, 135, 139, 134, 134, 134, 137, 134, 136, 134, 134, 134, 136, 135, 136, 134, 137, 140, 134, 134, 136]
4458 [126, 126, 127, 126, 128, 131, 126, 130, 126, 126, 126, 126, 129, 132, 126, 127, 131, 126, 126, 126, 129, 126, 128, 126, 126, 126, 128, 127, 128, 126, 129, 132, 126, 126, 128]
4178 [118, 118, 119, 118, 120, 123, 118, 122, 118, 118, 118, 118, 121, 124, 118, 119, 123, 118, 118, 118, 121, 118, 120, 118, 118, 118, 120, 119, 120, 118, 121, 124, 118, 118, 120]
3898 [110, 110, 111, 110, 112, 115, 110, 114, 110, 110, 110, 110, 113, 116, 110, 111, 115, 110, 110, 110, 113, 110, 112, 110, 110, 110, 112, 111, 112, 110, 113, 116, 110, 110, 112]
3618 [102, 102, 103, 102, 104, 107, 102, 106, 102, 102, 102, 102, 105, 108, 102, 103, 107, 102, 102, 102, 105, 102, 104, 102, 102, 102, 104, 103, 104, 102, 105, 108, 102, 102, 104]
3338 [94, 94, 95, 94, 96, 99, 94, 98, 94, 94, 94, 94, 97, 100, 94, 95, 99, 94, 94, 94, 97, 94, 

7785 [222, 222, 223, 222, 222, 222, 222, 226, 222, 222, 222, 222, 222, 228, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 224, 222, 222, 224, 222, 222, 222]
7505 [214, 214, 215, 214, 214, 214, 214, 218, 214, 214, 214, 214, 214, 220, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 216, 214, 214, 216, 214, 214, 214]
7225 [206, 206, 207, 206, 206, 206, 206, 210, 206, 206, 206, 206, 206, 212, 206, 206, 206, 206, 206, 206, 206, 206, 206, 206, 206, 206, 206, 206, 208, 206, 206, 208, 206, 206, 206]
6945 [198, 198, 199, 198, 198, 198, 198, 202, 198, 198, 198, 198, 198, 204, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 200, 198, 198, 200, 198, 198, 198]
6665 [190, 190, 191, 190, 190, 190, 190, 194, 190, 190, 190, 190, 190, 196, 190, 190, 190, 190, 190, 190, 190, 190, 190, 190, 190, 190, 190, 190, 192, 190, 190, 192, 190, 190, 190]
6385 [182, 182, 183, 182, 182, 182, 182, 186, 182, 182, 182, 182, 182, 188, 182, 182, 182, 182,

## Merge & 检查是否正确

In [43]:
df = pd.merge(df_origin, df_stKNNs[0][['utc_time', 'stationId', 'PM2.5']], how='left', suffixes=("", "_pred"),
                  left_on=['utc_time', 'stationId'],
                  right_on=['utc_time', 'stationId'],
             )

df = pd.merge(df, df_stKNNs[1][['utc_time', 'stationId', 'PM10']], how='left', suffixes=("", "_pred"),
                  left_on=['utc_time', 'stationId'],
                  right_on=['utc_time', 'stationId'],
             )

df = pd.merge(df, df_stKNNs[2][['utc_time', 'stationId', 'O3']], how='left', suffixes=("", "_pred"),
                  left_on=['utc_time', 'stationId'],
                  right_on=['utc_time', 'stationId'],
             )

tmp = df[pd.notnull(df[target])]
mean_absolute_error(tmp[target], tmp[target+'_pred'])

for target in target_cols:
    df[target] = df[target + '_pred']

# df = df[['utc_time', 'PM2.5', 'PM10', 'O3', 'stationId']]
df.to_csv("../input/beijing_1701_1803_impute.csv", index=False)

## Evalution (手动选择某一站点，设置一天空值)

In [27]:
# condition = (df.stationId=="tiantan_aq") & (df.utc_time>='2017-05-01 00:00:00') & (df.utc_time<='2017-05-01 23:00:00')
# tmp = df[['utc_time','PM10', 'PM10_pred']][condition]
# tmp = tmp.sort_values(by='utc_time')
# # tmp
# tmp.plot(x='utc_time', y=['PM10', 'PM10_pred'], figsize=(16,6), style={'PM10': '-', 'PM10_pred': ':'})

# tmp = df[condition]
# tmp = tmp[pd.notnull(tmp[target])]
# MAE = mean_absolute_error(tmp[target], tmp[target+'_pred'])
# print(MAE)

In [36]:
df = pd.merge(df, df_station, 'left', left_on='stationId', right_on='Station_ID')

df.longitude = df.longitude.round(1)
df.latitude = df.latitude.round(1)
df = pd.merge(df, df_meo, 'left', 
              left_on=['utc_time', 'longitude', 'latitude'], 
              right_on=['utc_time', 'longitude', 'latitude']
             )

_, bins = pd.cut(df['wind_direction'], bins=16, retbins=True)
df['wind_direction'] = pd.cut(df['wind_direction'], bins=16, labels=False)

le_wind_direction = LabelEncoder()
le_wind_direction.fit(df.station_type)
df.station_type = le_wind_direction.transform(df.station_type)


le_stationId = LabelEncoder()
le_stationId.fit(df.stationId)
df.stationId = le_stationId.transform(df.stationId)

# df = pd.get_dummies(df, columns=['stationId'])
df['year'] = df.utc_time.dt.year
df['month'] = df.utc_time.dt.month
df['day'] = df.utc_time.dt.day
df['hour'] = df.utc_time.dt.hour
df['weekday'] = df.utc_time.dt.weekday

,stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
0,aotizhongxin_aq,2017-01-01 22:00:00,453.0,467.0,156.0,7.2,3.0,9.0
1,aotizhongxin_aq,2017-01-01 23:00:00,417.0,443.0,143.0,6.8,2.0,8.0
2,aotizhongxin_aq,2017-01-02 00:00:00,395.0,467.0,141.0,6.9,3.0,8.0
3,aotizhongxin_aq,2017-01-02 01:00:00,420.0,484.0,139.0,7.4,3.0,9.0
4,aotizhongxin_aq,2017-01-02 02:00:00,453.0,520.0,157.0,7.6,4.0,9.0
5,aotizhongxin_aq,2017-01-02 03:00:00,429.0,NaN,141.0,6.5,3.0,9.0
6,aotizhongxin_aq,2017-01-02 04:00:00,211.0,NaN,110.0,3.3,NaN,11.0
7,aotizhongxin_aq,2017-01-02 05:00:00,116.0,NaN,87.0,2.2,4.0,13.0
8,aotizhongxin_aq,2017-01-02 06:00:00,51.0,NaN,58.0,1.3,26.0,14.0
9,aotizhongxin_aq,2017-01-02 07:00:00,38.0,NaN,55.0,1.1,28.0,14.0


In [52]:
df = pd.read_csv("../input/bj_meo_0330_0428.csv")

In [57]:
del df['id']
del df['weather']

In [56]:
df_meo

,stationName,longitude,latitude,utc_time,temperature,pressure,humidity,wind_direction,wind_speed/kph
0,beijing_grid_000,115.0,39.0,2017-01-01 00:00:00,-5.47,984.73,76.60,53.71,3.53
1,beijing_grid_001,115.0,39.1,2017-01-01 00:00:00,-5.53,979.33,75.40,43.59,3.11
2,beijing_grid_002,115.0,39.2,2017-01-01 00:00:00,-5.70,963.14,71.80,0.97,2.75
3,beijing_grid_003,115.0,39.3,2017-01-01 00:00:00,-5.88,946.94,68.20,327.65,3.84
4,beijing_grid_004,115.0,39.4,2017-01-01 00:00:00,-5.34,928.80,58.81,317.85,6.14
5,beijing_grid_005,115.0,39.5,2017-01-01 00:00:00,-4.81,910.66,49.43,313.46,8.52
6,beijing_grid_006,115.0,39.6,2017-01-01 00:00:00,-4.98,889.48,45.64,309.89,10.05
7,beijing_grid_007,115.0,39.7,2017-01-01 00:00:00,-5.49,866.77,44.66,306.63,11.16
8,beijing_grid_008,115.0,39.8,2017-01-01 00:00:00,-6.17,853.42,46.57,299.83,11.28
9,beijing_grid_009,115.0,39.9,2017-01-01 00:00:00,-7.17,858.79,54.29,281.74,9.98


In [59]:
df_grid_station = pd.read_csv("../input/Beijing_grid_weather_station")

FileNotFoundError: File b'../input/Beijing_grid_weather_station' does not exist